In [32]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
import tqdm

In [2]:
df = pd.read_csv('total_data(2016제외).csv')

### xdata로 모든 column을 넣어서 Elastic
- elasticnet으로 모든 column을 넣어 모델생성

In [3]:
# 필요 패키지 로드
#from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error

In [4]:
# y값인 q1-q5가 결측인 2020년 데이터 제거
a = df[0:-82]
a

,jur_stn,year,report_sx_남성,report_sx_불상,report_sx_여성,report_sx_total,vio_cnt,mur_rob_cnt,car_cnt,5m_crm_yn,...,single_f_2030,single_f_4050,single_f_o60,adult,bell,q1,q2,q3,q4,q5
0,서울중부경찰서,2017(상반기),13404.0,3239.0,5931.0,22574.0,1324.0,0.0,1412.0,3275.0,...,2128.0,1230.5,1635.5,0.0,0.0,75.2,76.5,69.9,65.9,72.6
1,서울종로경찰서,2017(상반기),11579.0,2560.0,5534.0,19674.0,1102.0,0.0,867.0,2914.0,...,2703.0,1378.5,1782.5,2.0,2.0,81.6,82.4,69.1,62.7,72.2
2,서울남대문경찰서,2017(상반기),9093.0,1855.0,3551.0,14499.0,949.0,0.0,736.0,2506.0,...,2128.0,1230.5,1635.5,0.0,0.0,77.3,79.3,68.8,61.0,71.7
3,서울서대문경찰서,2017(상반기),18441.0,10029.0,10298.0,38769.0,1924.0,0.0,1788.0,4267.0,...,12420.0,4496.0,7066.0,0.0,0.0,78.0,80.0,67.0,66.3,71.4
4,서울혜화경찰서,2017(상반기),10829.0,2989.0,5038.0,18857.0,1127.0,0.0,899.0,2864.0,...,2703.0,1378.5,1782.5,0.0,0.0,78.6,80.6,70.7,65.2,75.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,창원서부경찰서,2019(하반기),12630.0,2990.0,7151.0,22771.0,1460.0,2.0,1964.0,2142.0,...,2900.0,4492.0,5574.0,1.0,1.0,79.1,79.5,68.5,71.2,76.0
242,마산중부경찰서,2019(하반기),12223.0,2552.0,6489.0,21264.0,1265.0,5.0,1584.0,1369.0,...,2216.0,2837.0,7186.0,3.0,3.0,80.8,80.0,70.8,73.2,76.5
243,마산동부경찰서,2019(하반기),15177.0,3190.0,7647.0,26014.0,1715.0,8.0,1910.0,1920.0,...,1496.0,2939.0,6166.0,1.0,1.0,75.0,76.5,69.4,68.9,73.1
244,진주경찰서,2019(하반기),21840.0,5262.0,11455.0,38557.0,2552.0,0.0,3570.0,2876.0,...,6026.0,5205.0,11823.0,8.0,8.0,78.7,79.6,70.3,74.6,75.6


In [5]:
# 경찰서와 연도 데이터 제거
a.drop(columns = ['jur_stn', 'year'], inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [6]:
a_ = a.drop(columns = ['q1', 'q2', 'q3', 'q4', 'q5'])

In [7]:
a_

,report_sx_남성,report_sx_불상,report_sx_여성,report_sx_total,vio_cnt,mur_rob_cnt,car_cnt,5m_crm_yn,theft_violence_x,robber_murder_x,...,single_2030,single_4050,single_o60,single_f_합계,single_f_u20,single_f_2030,single_f_4050,single_f_o60,adult,bell
0,13404.0,3239.0,5931.0,22574.0,1324.0,0.0,1412.0,3275.0,572.0,4.0,...,4310.5,2954.5,2567.5,5169.0,175.0,2128.0,1230.5,1635.5,0.0,0.0
1,11579.0,2560.0,5534.0,19674.0,1102.0,0.0,867.0,2914.0,525.0,2.0,...,5483.5,3220.5,2861.5,6022.0,158.0,2703.0,1378.5,1782.5,2.0,2.0
2,9093.0,1855.0,3551.0,14499.0,949.0,0.0,736.0,2506.0,413.0,0.0,...,4310.5,2954.5,2567.5,5169.0,175.0,2128.0,1230.5,1635.5,0.0,0.0
3,18441.0,10029.0,10298.0,38769.0,1924.0,0.0,1788.0,4267.0,674.0,0.0,...,23040.0,9687.0,9894.0,24527.0,545.0,12420.0,4496.0,7066.0,0.0,0.0
4,10829.0,2989.0,5038.0,18857.0,1127.0,0.0,899.0,2864.0,427.0,4.0,...,5483.5,3220.5,2861.5,6022.0,158.0,2703.0,1378.5,1782.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,12630.0,2990.0,7151.0,22771.0,1460.0,2.0,1964.0,2142.0,363.0,2.0,...,8529.0,10946.0,8555.0,13016.0,50.0,2900.0,4492.0,5574.0,1.0,1.0
242,12223.0,2552.0,6489.0,21264.0,1265.0,5.0,1584.0,1369.0,151.0,3.0,...,5895.0,6580.0,10395.0,12431.0,192.0,2216.0,2837.0,7186.0,3.0,3.0
243,15177.0,3190.0,7647.0,26014.0,1715.0,8.0,1910.0,1920.0,312.0,6.0,...,3791.0,7160.0,9299.0,10643.0,42.0,1496.0,2939.0,6166.0,1.0,1.0
244,21840.0,5262.0,11455.0,38557.0,2552.0,0.0,3570.0,2876.0,537.0,0.0,...,15178.0,11882.0,16273.0,23362.0,308.0,6026.0,5205.0,11823.0,8.0,8.0


In [8]:
a_1 = a_[['report_sx_여성', 'vio_cnt', 'theft_violence_x', 'robber_murder_x',
       'theft_violence_y', 'robber_murder_y', '가해_10대이하', '피해_80대이상', 'for_합계',
       'popu_2030', 'popu_o60', 'for_u20', 'for_2030', 'for_4050', 'for_o60',
       'single_합계', 'single_2030', 'single_4050', 'single_o60', 'single_f_합계',
       'single_f_4050', 'single_f_o60']]

In [9]:
a_2 = a_[['report_sx_여성', 'vio_cnt', 'theft_violence_x', 'robber_murder_x',
       'theft_violence_y', 'robber_murder_y', '가해_10대이하', '피해_80대이상', 'for_합계',
       'popu_o60', 'for_u20', 'for_2030', 'for_4050', 'for_o60', 'single_합계',
       'single_4050', 'single_o60', 'single_f_합계', 'single_f_4050',
       'single_f_o60']]

In [10]:
a_3 = a_[['mur_rob_cnt', '5m_crm_yn', 'theft_violence_x', 'robber_murder_x',
       'traffic_x', 'theft_violence_y', 'robber_murder_y', '가해_70대',
       '가해_80대이상', '피해_70대', 'for_합계', 'for_2030', 'for_4050', 'for_o60',
       'single_합계', 'single_2030', 'single_4050', 'single_f_2030', 'adult',
       'bell']]

In [11]:
a_4 = a_[['mur_rob_cnt', 'car_cnt', 'theft_violence_x', '가해_여', 'cctv', 'for_합계',
       'for_2030', 'for_4050', 'for_o60', 'single_합계', 'single_4050']]

In [12]:
a_5 = a_[['report_sx_여성', 'vio_cnt', 'theft_violence_x', 'robber_murder_x',
       'theft_violence_y', 'robber_murder_y', '가해_여', '가해_10대이하', '피해_80대이상',
       'for_합계', 'popu_2030', 'popu_o60', 'for_u20', 'for_2030', 'for_4050',
       'for_o60', 'single_합계', 'single_2030', 'single_4050', 'single_o60',
       'single_f_합계', 'single_f_4050', 'single_f_o60']]

In [13]:
# StandardScaling
scaler1 = StandardScaler()
scaler1.fit(a_1) # scaler에 xdata 학습
a_s1 = scaler1.transform(a_1)
a_s1

array([[-1.08891897, -1.08002514, -0.40220751, ..., -1.40869518,
        -1.70872887, -1.55004946],
       [-1.17135971, -1.26140432, -0.54579805, ..., -1.32273075,
        -1.62995962, -1.50210803],
       [-1.58314811, -1.38640889, -0.88797125, ..., -1.40869518,
        -1.70872887, -1.55004946],
       ...,
       [-0.7325756 , -0.76056902, -1.19653815, ..., -0.85703135,
        -0.79942306, -0.07250741],
       [ 0.05819103, -0.07672048, -0.50913663, ...,  0.42477579,
         0.406598  ,  1.77242259],
       [-1.01686119, -0.98198234, -1.57842789, ..., -0.98925329,
        -0.98410501, -0.47528071]])

In [14]:
# StandardScaling
scaler2 = StandardScaler()
scaler2.fit(a_2) # scaler에 xdata 학습
a_s2 = scaler2.transform(a_2)
a_s2

array([[-1.08891897, -1.08002514, -0.40220751, ..., -1.40869518,
        -1.70872887, -1.55004946],
       [-1.17135971, -1.26140432, -0.54579805, ..., -1.32273075,
        -1.62995962, -1.50210803],
       [-1.58314811, -1.38640889, -0.88797125, ..., -1.40869518,
        -1.70872887, -1.55004946],
       ...,
       [-0.7325756 , -0.76056902, -1.19653815, ..., -0.85703135,
        -0.79942306, -0.07250741],
       [ 0.05819103, -0.07672048, -0.50913663, ...,  0.42477579,
         0.406598  ,  1.77242259],
       [-1.01686119, -0.98198234, -1.57842789, ..., -0.98925329,
        -0.98410501, -0.47528071]])

In [15]:
# StandardScaling
scaler3 = StandardScaler()
scaler3.fit(a_3) # scaler에 xdata 학습
a_s3 = scaler3.transform(a_3)
a_s3

array([[-0.68019192, -0.53368129, -0.40220751, ..., -0.97165206,
        -0.59342342, -0.59342342],
       [-0.68019192, -0.71416836, -0.54579805, ..., -0.87776189,
         0.29943384,  0.29943384],
       [-0.68019192, -0.91815375, -0.88797125, ..., -0.97165206,
        -0.59342342, -0.59342342],
       ...,
       [ 1.90901058, -1.21113276, -1.19653815, ..., -1.0748496 ,
        -0.14699479, -0.14699479],
       [-0.68019192, -0.733167  , -0.50913663, ..., -0.33515836,
         2.97800563,  2.97800563],
       [-0.68019192, -1.49411249, -1.57842789, ..., -1.02814945,
         0.74586247,  0.74586247]])

In [16]:
# StandardScaling
scaler4 = StandardScaler()
scaler4.fit(a_4) # scaler에 xdata 학습
a_s4 = scaler4.transform(a_4)
a_s4

array([[-0.68019192, -0.92567779, -0.40220751, ..., -0.38911982,
        -1.38252074, -1.64561528],
       [-0.68019192, -1.4252536 , -0.54579805, ..., -0.45704651,
        -1.29327693, -1.58609783],
       [-0.68019192, -1.54533513, -0.88797125, ..., -0.38911982,
        -1.38252074, -1.64561528],
       ...,
       [ 1.90901058, -0.46918467, -1.19653815, ..., -0.59093101,
        -0.85594629, -0.70463545],
       [-0.68019192,  1.05245908, -0.50913663, ..., -0.55745988,
         0.36027682,  0.35191116],
       [-0.68019192, -0.66901499, -1.57842789, ..., -0.5358021 ,
        -0.82352669, -0.66010924]])

In [17]:
# StandardScaling
scaler5 = StandardScaler()
scaler5.fit(a_5) # scaler에 xdata 학습
a_s5 = scaler5.transform(a_5)
a_s5

array([[-1.08891897, -1.08002514, -0.40220751, ..., -1.40869518,
        -1.70872887, -1.55004946],
       [-1.17135971, -1.26140432, -0.54579805, ..., -1.32273075,
        -1.62995962, -1.50210803],
       [-1.58314811, -1.38640889, -0.88797125, ..., -1.40869518,
        -1.70872887, -1.55004946],
       ...,
       [-0.7325756 , -0.76056902, -1.19653815, ..., -0.85703135,
        -0.79942306, -0.07250741],
       [ 0.05819103, -0.07672048, -0.50913663, ...,  0.42477579,
         0.406598  ,  1.77242259],
       [-1.01686119, -0.98198234, -1.57842789, ..., -0.98925329,
        -0.98410501, -0.47528071]])

In [18]:
# x데이터 설정 2017, 2018 데이터를 학습용, 2019년 데이터를 검증용 데이터셋으로 설정

xtrain1 = a_s2[:-82]
xtest1 = a_s2[-82:]
xtrain2 = a_s2[:-82]
xtest2 = a_s2[-82:]
xtrain3 = a_s3[:-82]
xtest3 = a_s3[-82:]
xtrain4 = a_s4[:-82]
xtest4 = a_s4[-82:]
xtrain5 = a_s5[:-82]
xtest5 = a_s5[-82:]

In [19]:
# y데이터 설정 2017, 2018 데이터를 학습용, 2019년 데이터를 검증용 데이터셋으로 설정

train = a[:-82]
test = a[-82:]

ytrain1 = train['q1']
ytrain2 = train['q2']
ytrain3 = train['q3']
ytrain4 = train['q4']
ytrain5 = train['q5']

ytest1 = test['q1']
ytest2 = test['q2']
ytest3 = test['q3']
ytest4 = test['q4']
ytest5 = test['q5']

In [20]:
a_1.columns

Index(['report_sx_여성', 'vio_cnt', 'theft_violence_x', 'robber_murder_x',
       'theft_violence_y', 'robber_murder_y', '가해_10대이하', '피해_80대이상', 'for_합계',
       'popu_2030', 'popu_o60', 'for_u20', 'for_2030', 'for_4050', 'for_o60',
       'single_합계', 'single_2030', 'single_4050', 'single_o60', 'single_f_합계',
       'single_f_4050', 'single_f_o60'],
      dtype='object')

In [21]:
# 그리드 서치 패키지
from sklearn.model_selection import GridSearchCV

In [22]:
# 최적 성능을 내는 lasso의 alpha값을 얻기 위해 param_grid 생성

param_grid = {'l1_ratio': np.linspace(0.01, 1.0, 100), 'alpha' : np.linspace(0.001, 1.0, 1000)}

#그리드 서치 설정
#grid_search = GridSearchCV(ElasticNet(), param_grid = param_grid, cv = 10, n_jobs = -1, scoring ='r2')
grid_search = GridSearchCV(ElasticNet(), param_grid = param_grid, cv = 10, n_jobs = -1, scoring ='neg_mean_absolute_error')

#### q1 절도폭력 

In [23]:
# 그리드 서치 후 최고 성능의 모델을 ela1에 저장

grid_search.fit(xtrain1, ytrain1)
ela1 = grid_search.best_estimator_

In [33]:
# MAE 출력

y_pred1 = ela1.predict(xtest1)
mean_absolute_error(ytest1, y_pred1)

2.828633854763776

In [34]:
# 결과

print('alpha =', ela1.alpha)
print('l1_ratio =', ela1.l1_ratio)
print(ela1.coef_) # Ridge 회귀분석으로 나온 weghit값
# print('feature 갯수:', len(a_1.columns[ela1.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_1.columns[ela1.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_1.columns[ela1.coef_.argmin()])

alpha = 0.035
l1_ratio = 1.0
[-0.         -1.92513667  1.99959511  0.39067922 -0.38238458 -0.06189105
 -0.3995936  -0.         -0.          3.32165242  1.27740873 -1.60078624
 -0.         -1.38947597  0.         -1.65513253 -2.3358609   1.21641297
  0.          0.        ]
가장 강한 양의 상관관계:  popu_2030 
가장 강한 음의 상관관계:  single_2030


In [52]:
# q1 관련 feature
print('상위 30개 feature\n', a_1.columns[sorted(range(len(ela1.coef_)), key=lambda i: abs(ela1.coef_[i]))[-30:]])

상위 30개 feature
 Index(['report_sx_여성', '피해_80대이상', 'for_합계', 'for_2030', 'for_o60',
       'single_o60', 'single_f_합계', 'robber_murder_y', 'theft_violence_y',
       'robber_murder_x', '가해_10대이하', 'single_4050', 'popu_o60', 'for_4050',
       'for_u20', 'single_합계', 'vio_cnt', 'theft_violence_x', 'single_2030',
       'popu_2030'],
      dtype='object')


#### q2 강도살인

In [35]:
# 그리드 서치 후 최고 성능의 모델을 ela2에 저장

grid_search.fit(xtrain2, ytrain2)
ela2 = grid_search.best_estimator_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 245.36067005190162, tolerance: 0.18037177439024385
  model = cd_fast.enet_coordinate_descent(


In [36]:
# MAE 출력

y_pred2 = ela2.predict(xtest2)
mean_absolute_error(ytest2, y_pred2)

2.8776996808012747

In [37]:
# 결과

print('alpha =', ela2.alpha)
print('l1_ratio =', ela2.l1_ratio)
print(ela2.coef_) # Ridge 회귀분석으로 나온 weghit값
# print('feature 갯수:', len(a_2.columns[ela2.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_2.columns[ela2.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_2.columns[ela2.coef_.argmin()])

alpha = 0.001
l1_ratio = 0.41000000000000003
[ 0.23081015 -2.5336387   2.14846718  0.38813378 -0.73377886 -0.15195375
 -0.82684201  0.00951223  1.9697012   5.63812435  1.81774899 -2.52933276
  3.35558772 -6.0566765   5.98283584 -4.73478705 -6.49852735 -4.98164327
  2.7168325   3.36179636]
가장 강한 양의 상관관계:  single_합계 
가장 강한 음의 상관관계:  single_o60


In [38]:
# q2 관련 feature
print('상위 30개 feature\n', a_2.columns[sorted(range(len(ela2.coef_)), key=lambda i: abs(ela2.coef_[i]))[-30:]])

상위 30개 feature
 Index(['피해_80대이상', 'robber_murder_y', 'report_sx_여성', 'robber_murder_x',
       'theft_violence_y', '가해_10대이하', 'for_u20', 'for_합계', 'theft_violence_x',
       'for_2030', 'vio_cnt', 'single_f_4050', 'for_4050', 'single_f_o60',
       'single_4050', 'single_f_합계', 'popu_o60', 'single_합계', 'for_o60',
       'single_o60'],
      dtype='object')


#### q3 교통안전

In [39]:
# 그리드 서치 후 최고 성능의 모델을 lasso3에 저장

grid_search.fit(xtrain3, ytrain3)
ela3 = grid_search.best_estimator_

In [40]:
# MAE 출력

y_pred3 = ela3.predict(xtest3)
mean_absolute_error(ytest3, y_pred3)

2.384404333026005

In [41]:
# 결과

print('alpha =', ela3.alpha)
print('l1_ratio =', ela3.l1_ratio)
print(ela3.coef_) # Ridge 회귀분석으로 나온 weghit값
# print('feature 갯수:', len(a_3.columns[ela3.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_3.columns[ela3.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_3.columns[ela3.coef_.argmin()])

alpha = 0.139
l1_ratio = 1.0
[ 0.          0.          0.          0.         -0.35441841 -0.
 -0.          0.41455777  0.          0.45956824 -0.         -0.09380322
 -0.01563009 -0.         -0.         -0.         -0.59842525 -0.
 -0.         -0.        ]
가장 강한 양의 상관관계:  피해_70대 
가장 강한 음의 상관관계:  single_4050


In [42]:
# q3 관련 feature
print('상위 30개 feature\n', a_3.columns[sorted(range(len(ela3.coef_)), key=lambda i: abs(ela3.coef_[i]))[-30:]])

상위 30개 feature
 Index(['mur_rob_cnt', '5m_crm_yn', 'theft_violence_x', 'robber_murder_x',
       'theft_violence_y', 'robber_murder_y', '가해_80대이상', 'for_합계', 'for_o60',
       'single_합계', 'single_2030', 'single_f_2030', 'adult', 'bell',
       'for_4050', 'for_2030', 'traffic_x', '가해_70대', '피해_70대', 'single_4050'],
      dtype='object')


#### q4 법질서 준수도

In [43]:
# 그리드 서치 후 최고 성능의 모델을 lasso4에 저장

grid_search.fit(xtrain4, ytrain4)
ela4 = grid_search.best_estimator_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 587.0294348000011, tolerance: 0.27214904878048785
  model = cd_fast.enet_coordinate_descent(


In [44]:
# MAE 출력

y_pred4 = ela4.predict(xtest4)
mean_absolute_error(ytest4, y_pred4)

2.7654490051814204

In [45]:
# 결과

print('alpha =', ela4.alpha)
print('l1_ratio =', ela4.l1_ratio)
print(ela4.coef_) # Ridge 회귀분석으로 나온 weghit값
# print('feature 갯수:', len(a_4.columns[ela4.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_4.columns[ela4.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_4.columns[ela4.coef_.argmin()])

alpha = 0.001
l1_ratio = 1.0
[ 1.40559904  3.30924757 -3.89413231  1.32492283  0.84844733  9.13908274
 -3.42138732 -6.29437051 -0.08955156  1.46688553 -2.17008255]
가장 강한 양의 상관관계:  for_합계 
가장 강한 음의 상관관계:  for_4050


In [46]:
# q3 관련 feature
print('상위 30개 feature\n', a_4.columns[sorted(range(len(ela4.coef_)), key=lambda i: abs(ela4.coef_[i]))[-30:]])

상위 30개 feature
 Index(['for_o60', 'cctv', '가해_여', 'mur_rob_cnt', 'single_합계', 'single_4050',
       'car_cnt', 'for_2030', 'theft_violence_x', 'for_4050', 'for_합계'],
      dtype='object')


#### q5 전반적 안전도

In [47]:
# 그리드 서치 후 최고 성능의 모델을 lasso4에 저장

grid_search.fit(xtrain5, ytrain5)
ela5 = grid_search.best_estimator_

In [48]:
# MAE 출력

y_pred5 = ela5.predict(xtest5)
mean_absolute_error(ytest5, y_pred5)

2.577853340348487

In [49]:
# 결과

print('alpha =', ela5.alpha)
print('l1_ratio =', ela5.l1_ratio)
print(ela5.coef_) # Ridge 회귀분석으로 나온 weghit값
# print('feature 갯수:', len(a_5.columns[ela5.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_5.columns[ela5.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_5.columns[ela5.coef_.argmin()])

alpha = 0.013000000000000001
l1_ratio = 0.64
[-0.         -1.3092545   1.15423487  0.54381212 -0.97758616 -0.2871621
  1.4378743  -0.99055749 -0.09554399 -0.12246979  0.89607574  2.09486584
  1.02065254 -0.83483412 -0.         -0.71631227 -0.         -0.
 -2.25317822 -1.02427495  0.35839254  0.51197084 -0.        ]
가장 강한 양의 상관관계:  popu_o60 
가장 강한 음의 상관관계:  single_4050


In [50]:
print('feature: ', a_5.columns[ela5.coef_ != 0])

feature:  Index(['vio_cnt', 'theft_violence_x', 'robber_murder_x', 'theft_violence_y',
       'robber_murder_y', '가해_여', '가해_10대이하', '피해_80대이상', 'for_합계',
       'popu_2030', 'popu_o60', 'for_u20', 'for_2030', 'for_o60',
       'single_4050', 'single_o60', 'single_f_합계', 'single_f_4050'],
      dtype='object')


In [51]:
# q5 관련 feature
print('상위 30개 feature\n', a_5.columns[sorted(range(len(ela5.coef_)), key=lambda i: abs(ela5.coef_[i]))[-30:]])

상위 30개 feature
 Index(['report_sx_여성', 'for_4050', 'single_합계', 'single_2030', 'single_f_o60',
       '피해_80대이상', 'for_합계', 'robber_murder_y', 'single_f_합계', 'single_f_4050',
       'robber_murder_x', 'for_o60', 'for_2030', 'popu_2030',
       'theft_violence_y', '가해_10대이하', 'for_u20', 'single_o60',
       'theft_violence_x', 'vio_cnt', '가해_여', 'popu_o60', 'single_4050'],
      dtype='object')
